In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

data = pd.read_csv('/datasets/data.csv')
data['education'] = data['education'].str.lower() 
data.info()

#data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Добрый день!
Импортируем библиотеки pandas, pymystem3 и collection, которые нам потребуются далее.

Создаем переменную data в которую помещаем таблицу с данными, которую мы читаем методом .read_csv.

Для удобства приравниваем в нижнему регистру данные из столбца "education" чтобы не сильно отвлекало в процессе изучения данных.

Выводим общую информацию по данным в файле. Наблюдаем ярко выраженные расхождения в количестве по 2 столбцам.

Выводим на экран первые 10 строк для представление общей картины.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
data['days_employed'].isna().sum()
data['total_income'].isna().sum()


data_whith_none = data.loc[(data['days_employed'] == '') & (data['total_income'] == '')].count()

data = data.drop_duplicates()
data.groupby('income_type')['total_income'].describe()

data['days_employed'] = data['days_employed'].fillna(value = 0)
data['total_income'] = data['total_income'].fillna(value = 0)

data['family_status_id'].describe()
data['family_status_id'].value_counts()

data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)
data['dob_years'] = data['dob_years'].replace(0, data['dob_years'].mean())


def days_employed_fill (row):
    income_type_row = row['income_type']
    days_employed_row = row['days_employed']
    if income_type_row == 'пенсионер' and days_employed_row == 0:
        return 365213.306266
    elif income_type_row == 'студент' and days_employed_row == 0:
        return -578.751554
    elif income_type_row == 'безработный' and days_employed_row == 0:
        return 366413.652744
    elif income_type_row == 'в декрете' and days_employed_row == 0:
        return -3296.759962
    elif income_type_row == 'госслужащий' and days_employed_row == 0:
        return -2689.368353
    elif income_type_row == 'компаньон' and days_employed_row == 0:
        return 172357.950966
    elif income_type_row == 'предприниматель' and days_employed_row == 0:
        return -520.848083
    elif income_type_row == 'сотрудник' and days_employed_row == 0:
        return -1574.202821
    else:
        return days_employed_row

data['days_employed'] = data.apply(days_employed_fill ,axis = 1)

def total_income_fill (row):
    income_type_row = row['income_type']
    total_income_row = row['total_income']
    if income_type_row == 'пенсионер' and total_income_row == 0:
        return 118514.486412
    elif income_type_row == 'студент' and total_income_row == 0:
        return 98201.625314
    elif income_type_row == 'безработный' and total_income_row == 0:
        return 131339.751676
    elif income_type_row == 'в декрете' and total_income_row == 0:
        return 53829.130729
    elif income_type_row == 'госслужащий' and total_income_row == 0:
        return 150447.935283
    elif income_type_row == 'компаньон' and total_income_row == 0:
        return 172357.950966
    elif income_type_row == 'предприниматель' and total_income_row == 0:
        return 499163.144947
    elif income_type_row == 'сотрудник' and total_income_row == 0:
        return 142594.396847
    else:
        return total_income_row

data['total_income'] = data.apply(total_income_fill, axis = 1)


#print(2174 / 21525 *100)

data['gender'].max()

data['gender'].loc[data['gender'] == 'XNA'].count()



#data['days_employed'].describe()

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


1

### Вывод
#В обоих столбцах ('days_employed' и 'total_income')2174 пропущенных значения. При помощи атрибута .loc, закоментированного далее, было выявлено что это одни и теже строки.

#Для заполнения пустых строк в этих двух столбцах были выбраны медианые значения т.к. средние 172 года трудового стажа выглядят совсем неправдоподобно. 

#Немного нарушена последовательность действий, предложенная в плане, и дубликаты были удалены на данном этапе. Сделано это для того чтобы при последующей замене пропущенных значений на усредненые или медианые значения не вызвать искажение итоговых данных(кол-во дупликатов 71).

#Заполнение пустых значенийв столбцах ('days_employed' и 'total_income') медиаными значениями для того чтобы не терять 10.09% данных в случае их полного удаления.


Предположительно артефакты:
#Методами "min\ max и unique" выявлены необычные случаи по ряду столбцов
#нулевые значения в графе с возрастом в количестве 101 случия; количество детей -1 и 20 (47 и 76 случия соответственно); непонятный пол "XNA" - 1 случай.

##
Пропуски в графах с трудовым стажем и ежемесячным доходом были заполнены нулевыми значениями.

После чего было написаное две функции, которые, ориентируясь на столбец с типом занятости проставляет в нулевые значения по столбцам значения 50% выборки(выявлены ранее методом describe()). 

Методами .describe() и .value_counts() были выявлены 0-е значения в столбце с возрастом клиентов(заменены на средний возраст т.к. далее в расчетах не участвует), количество детей "-1" заменено на "1", а "20" на "2"(предположительно была допущена ошибка связанная с человеческим фактором)



### Замена типа данных

In [3]:
data['total_income'] = data['total_income'].astype(int)
data['days_employed'] = data['days_employed'].astype(int)
data['dob_years'] = data['dob_years'].astype(int)


data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод


#Замена вещественного типа данных на целочисленный.
#to_numeric не подходил т.к. при его использовании числа будут иметь тип float (как у исходных данных).

##
Произведена целочисленная замена в столбцах с общим трудовым стажем, ежемесячным доходом и возрастом.


### Обработка дубликатов

In [4]:
data.duplicated().sum()

0

### Вывод

Дубликаты были обработаны ранее в части "Обработка пропусков".
##
Произведен подсчет дупликатов в таблице(значение "0").

### Лемматизация

In [5]:
def text_for_lemmatize(row):
    return m.lemmatize(row['purpose'])  

data['lemmas'] = data.apply(text_for_lemmatize, axis=1) 

### Вывод

#Проводим лемматизацию столбца "purpose" для приведения слов к единообразию при помощи функции text_for_lemmatize, которая принимает строку как аргумент. После чего возвращает значение столбща.

#Добавляем новый столбец в основную таблицу с данными для записи подверженного лематизации столбца "purpose". аргументом выступает функция для лемматизации выше.


### Категоризация данных

In [6]:
def purpose_category(row):   
    lemmas = row['lemmas']
    if 'жилье' in lemmas or 'недвижимость' in lemmas or 'квартира' in lemmas:
        return 'недвижимость'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'образование' in lemmas:
        return 'образование'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    
data['purpose_category'] = data.apply(purpose_category, axis = 1)

def total_income_category(row):
    row = row['total_income']
    if row < 145000:
        return 'ниже среднего'
    elif 145001 < row < 300000:
        return 'средний'
    else:
        return 'выше среднего'
    
data['total_income_category'] = data.apply(total_income_category, axis = 1)  


def children_category (row):  
    row = row['children']
    if row == 0:
        return 'без детей'
    elif 1 <= row <= 2:
        return 'семья с детьми'
    elif 3 <= row < 6:
        return 'многодетные'
    else:
        return 'over многодетные'
    
data['children_category'] = data.apply(children_category, axis = 1) 



### Вывод

#Пишем функцию для категоризации нового столбца с леммами. Категории были выявлены ранее методом .unique(). 

#Добавление столбца в общую таблицу с категориями целей получения кредита. Аргументом выступает функция для категоризации полученных ранее лемм.

#Функция для категоризации уровня дохода. Три категории: "ниже среднего", "средний" и "выше среднего", были выбраны мною как наиболее часто встречающиеся для описания доходов. Начальное пороговое значение было выбрано исходя из медианого значения с шагом Х2.

#Добавление столбца в общую таблицу с данными с категориями уровня доходов. Аргументом выступает функция для категоризации уровня дохода.


#Пишем функцию для разбивки количества детей на категории.

#Добавляем в общую таблицу столбец с категориями по количеству детей в семье.


In [7]:
data['total_income_category'].value_counts()

ниже среднего    11130
средний           8841
выше среднего     1483
Name: total_income_category, dtype: int64

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
data_pivot_children = data.pivot_table(index = ['children_category'], columns='debt', values= 'total_income', aggfunc= 'count')

data_pivot_children['percentage_return'] = data_pivot_children[0] / (data_pivot_children[0] + data_pivot_children[1]) * 100

print(data_pivot_children)

debt                   0     1  percentage_return
children_category                                
без детей          13028  1063          92.456178
многодетные          349    31          91.842105
семья с детьми      6336   647          90.734641


### Вывод

#Построение сводной таблицы для определения связи между наличием детей и возвратом денег в срок. Группируем по столбцу с категориями кол-ва детей, столбец с значениями кто отдавал кредит в срок,а кто нет, значения - количество людей по столбцу с доходом.

#Добавление к сводной таблице столбца с процентным соотношением случаев возврата денег вовремя к общему числу по категории наличии детей.

#Самый высокий процент возврата у семей без детей т.к. им гораздо проще следить за домашним бюджетом и как следствие выплачивать кредит вовремя.
#Семьи с большим количеством детей хоть и обращаются за кредитами реже, но и имеют самый низкий процент по возврату в срок.


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
data_family_status = data.pivot_table(index = ['family_status_id', 'family_status'], columns = 'debt', values= 'total_income', aggfunc='count')

data_family_status['percentage_return'] = data_family_status[0] / (data_family_status[0] + data_family_status[1]) * 100

data_family_status

,debt,0,1,percentage_return
family_status_id,family_status,,,
0,женат / замужем,11408,931,92.454818
1,гражданский брак,3763,388,90.652855
2,вдовец / вдова,896,63,93.430657
3,в разводе,1110,85,92.887029
4,Не женат / не замужем,2536,274,90.249110


### Вывод

#Построение сводной таблицы для определения связи между семейным положениеми возвратом денег в срок. Группируем по столбцам с id семейного статуса и самому семейному статусу, столбец с значениями кто отдавал кредит в срок,а кто нет, значения - количество людей по столбцу с доходом.

#Добавление к сводной таблице столбец с процентным соотношением случаев возврата денег вовремя к общему числу по категории семейного статуса.

#Самый высокий процент возврата кредита показала категория "вдовец / вдова". Судя по общему числу заявок на кредиты от данной группы людей, в денежных средставах они нуждаются меньше всех прочих и как следствие испытывают меньше всего проблем с их возвратом.



- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
data_total_income_category = data.pivot_table(index=['total_income_category'], columns='debt', values='total_income', aggfunc='count')

data_total_income_category['percentage_return'] = data_total_income_category[0] / (data_total_income_category[0] + data_total_income_category[1]) * 100
data_total_income_category.sort_values('percentage_return', ascending= False)

debt,0,1,percentage_return
total_income_category,,,
выше среднего,1377,106,92.852326
средний,8143,698,92.104966
ниже среднего,10193,937,91.581312


### Вывод

#Построение сводной таблицы для определения связи между уровнем дохода и возвратом денег в срок.Группируем по столбцу с категорией уровня доходов, столбец с значениями кто отдавал кредит в срок,а кто нет, значения - количество людей по столбцу с доходом.

#Самый высокий процент показала категория людей с уровнем дохода "выше среднего".

#Предполагаю, что людям с высоким уровнем дохода проще найти средства для погашения кредитов нежели двум другим категориям. Линейная прогрессия тому подтверждение.


- Как разные цели кредита влияют на его возврат в срок?

### Вывод

#Построение сводной таблицы для определения связи между целью получения кредита и возвратом денег в срок. Группируем по столбцу с целевыми категориями, столбец с значениями кто отдавал кредит в срок,а кто нет, значения - количество людей по столбцу с доходом.

#Добавление к сводной таблице столбца с процентным соотношением количества случаев возврата денег в срок к общему числу выданых по категории цели получения кредита.

#Самый высокий процент по возврату показала категория людей, которые берут кредит на недвижимость.
#Скорее всего так получается по причине того, что к покупке жилья люди подходят более осознано и вдумчиво.


### Шаг 4. Общий вывод

Если основываться вычислениях по заданным вопросам, то можно сделать следующие выводы: 
Наименьшее влияние на погашение кредита в срок оказывает уровень дохода кредитуемого.
Самое большое расхождение в рамках определенных категорий показала метрика с семейным статусом.
Ни по одной из категорий небыл выявлен случай не выплаты кредита в срок ниже вероятности в 89.47%.
